In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from dataloader import read_bci_data
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")

In [3]:
batch_size = 270

In [4]:
os.chdir("C:\\Users\\GL75\\OneDrive\\桌面\\深度學習\\Lab2")

In [5]:
_, _, X_test, y_test = read_bci_data()

(1080, 1, 2, 750) (1080,) (1080, 1, 2, 750) (1080,)


In [6]:
test_dataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [7]:
class EEGNet(nn.Module):
    def __init__(self, activation_fn = nn.ELU()):
        super().__init__()
        self.firstconv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = (1, 51), stride = (1, 1), padding = (0, 25), bias = False),
            nn.BatchNorm2d(16, eps = 1e-5, momentum = 0.1, affine = True, track_running_stats = True)
        )
        self.depthwiseConv = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size = (2, 1), stride = (1, 1), groups = 16, bias = False), # input channels = groups, meaning depthwise, a channel a filter
            nn.BatchNorm2d(32, eps = 1e-5, momentum = 0.1, affine = True, track_running_stats = True),
            activation_fn,
            nn.AvgPool2d(kernel_size = (1, 4), stride = (1, 4), padding = 0),
            nn.Dropout(p = 0.25)
        )
        self.separableConv = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size = (1, 15), stride = (1, 1), padding = (0, 7), bias = False),
            nn.BatchNorm2d(32, eps = 1e-5, momentum = 0.1, affine = True, track_running_stats = True),
            activation_fn,
            nn.AvgPool2d(kernel_size = (1, 8), stride = (1, 8), padding = 0),
            nn.Dropout(p = 0.25)
        )
        self.flatten = nn.Flatten()
        self.classify = nn.Linear(736, 2)
    def forward(self, X):
        output = self.firstconv(X)
        output = self.depthwiseConv(output)
        output = self.separableConv(output)
        output = self.flatten(output)
        output = self.classify(output)
        return output

In [8]:
def testing(model, test_dataloader, device):
    model.eval()
    accuracy = 0.0
    with torch.no_grad():
        for (X_batched, y_batched) in test_dataloader:
            X_batched = X_batched.to(device, dtype = torch.float)
            y_batched = y_batched.to(device, dtype = torch.long)
            prediction = model(X_batched)
            accuracy += prediction.max(dim = 1)[1].eq(y_batched).sum().item()
        accuracy = accuracy / len(test_dataloader.dataset) * 100
    return accuracy

In [9]:
model = EEGNet(nn.LeakyReLU())
model.load_state_dict(torch.load(os.path.join("D://DL_lab2_params/EEGNet", "LeakyReLU.pt"), map_location = device))
model.to(device)
acc = testing(model, test_dataloader, device)
print(f"The testing accuracy of the model is: {acc:.2f}%")

The testing accuracy of the model is: 87.31%
